## 경사하강을 이용한 행렬분해

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error

R = np.array([[4,np.NaN,np.NaN,2,np.NaN],
             [np.NaN,5,np.NaN,3,1],
             [np.NaN,np.NaN,3,4,4],
             [5,2,1,2,np.NaN]])
num_users, num_items = R.shape
k=3 # 잠재요인 차원

In [3]:
# P와 Q행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력
np.random.seed(1)

P = np.random.normal(scale=1./k, size=(num_users,k))
Q = np.random.normal(scale=1./k, size=(num_items,k))

In [4]:
P

array([[ 0.54144845, -0.2039188 , -0.17605725],
       [-0.35765621,  0.28846921, -0.76717957],
       [ 0.58160392, -0.25373563,  0.10634637],
       [-0.08312346,  0.48736931, -0.68671357]])

In [5]:
Q.T

array([[-0.1074724 , -0.36663042,  0.01407125,  0.38157457,  0.30028532],
       [-0.12801812, -0.05747607,  0.19427174,  0.30053024, -0.22790929],
       [ 0.37792315, -0.29261947, -0.36687306,  0.16749811, -0.04096341]])

In [6]:
def get_rmse(R,P,Q,non_zeros):
#     error=0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 행렬 R 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측행렬 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse



In [7]:
non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] >0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

for step in range(steps):
    for i,j,r in non_zeros:
        eij = r - np.dot(P[i,:], Q[j,:].T)
        
        P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:]-r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
        
    rmse = get_rmse(R,P,Q,non_zeros)
    if (step%50)==0:
        print('### iteration step :',step,'rmse :',rmse)

### iteration step : 0 rmse : 3.2388050277987723
### iteration step : 50 rmse : 0.4876723101369648
### iteration step : 100 rmse : 0.1564340384819247
### iteration step : 150 rmse : 0.07455141311978046
### iteration step : 200 rmse : 0.04325226798579314
### iteration step : 250 rmse : 0.029248328780878973
### iteration step : 300 rmse : 0.022621116143829466
### iteration step : 350 rmse : 0.019493636196525135
### iteration step : 400 rmse : 0.018022719092132704
### iteration step : 450 rmse : 0.01731968595344266
### iteration step : 500 rmse : 0.016973657887570753
### iteration step : 550 rmse : 0.016796804595895633
### iteration step : 600 rmse : 0.01670132290188466
### iteration step : 650 rmse : 0.01664473691247669
### iteration step : 700 rmse : 0.016605910068210026
### iteration step : 750 rmse : 0.016574200475705
### iteration step : 800 rmse : 0.01654431582921597
### iteration step : 850 rmse : 0.01651375177473524
### iteration step : 900 rmse : 0.01648146573819501
### iteration

In [8]:
pred_matrix = np.dot(P,Q.T)
np.round(pred_matrix,3)

array([[3.991, 0.897, 1.306, 2.002, 1.663],
       [6.696, 4.978, 0.979, 2.981, 1.003],
       [6.677, 0.391, 2.987, 3.977, 3.986],
       [4.968, 2.005, 1.006, 2.017, 1.14 ]])

In [9]:
print(non_zeros)

[(0, 0, 4.0), (0, 3, 2.0), (1, 1, 5.0), (1, 3, 3.0), (1, 4, 1.0), (2, 2, 3.0), (2, 3, 4.0), (2, 4, 4.0), (3, 0, 5.0), (3, 1, 2.0), (3, 2, 1.0), (3, 3, 2.0)]


## 컨텐츠 기반 필터링

In [73]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [74]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [75]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [76]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
movies_df.head()
pd.set_option('max_colwidth',100)

In [77]:
movies_df.info()
movies_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 53, ""name"": ""Thriller""}]",6.6,238,14.269792,"[{""id"": 5616, ""name"": ""united states\u2013mexico barrier""}, {""id"": 33649, ""name"": ""legs""}, {""id""...","El Mariachi just wants to play his guitar and carry on the family tradition. Unfortunately, the ..."
4799,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""name"": ""Romance""}]",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by the arrivals of their respective sisters.
4800,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""name"": ""Romance""}, {""...",7.0,6,1.444476,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""name"": ""love at first sight""}, {""id"": 2398, ""name"": ""...","""Signed, Sealed, Delivered"" introduces a dedicated quartet of civil servants in the Dead Letter ..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],"When ambitious New York attorney Sam is sent to Shanghai on assignment, he immediately stumbles ..."


In [78]:
movies_df[['keywords','genres']][:1]

,keywords,genres
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {..."


In [79]:
# literal_eval : 리스트 객체 list[(dic1,dic2)] 객체를 만들어 줌
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


In [80]:
movies_df[['keywords','genres']][:1]

,keywords,genres
0,"[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {..."


In [81]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[y['name'] for y in x])
movies_df[['keywords','genres']][:1]

,keywords,genres
0,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","[Action, Adventure, Fantasy, Science Fiction]"


In [82]:
print(movies_df['genres'][2])
print(movies_df['genres'][1740])

['Action', 'Adventure', 'Crime']
['Action', 'Adventure', 'Crime']


In [83]:
from sklearn.feature_extraction.text import CountVectorizer

# CounterVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x:(' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [84]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:5])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.12598816 0.16903085 0.3380617  ... 0.12598816 0.         0.        ]
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]]


In [85]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:5])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 [2195 1850 3316 ...  887 2544 4802]
 [ 102 2995   56 ... 3046 3045 2401]]


In [98]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 20)
similar_movies[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866 3112 4041  588 3337
  3378  281 1663 1464 1149 2839]]


,title,vote_average,vote_count
3337,The Godfather,8.4,5893
2731,The Godfather: Part II,8.3,3338
1663,Once Upon a Time in America,8.2,1069
1847,GoodFellas,8.2,3128
3866,City of God,8.1,1814
883,Catch Me If You Can,7.7,3795
281,American Gangster,7.4,1502
4041,This Is England,7.4,363
1243,Mean Streets,7.2,345
2839,Rounders,6.9,439


In [99]:
movies[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


#### 과제_1024_1
p.609~p.612 예제를 풀어보시고 컨텐츠 기반 필터링에 대한 영화 추천 방식에서 유의할 사항을 설명하세요.

In [100]:
# 가중 평점 구하기
# v : 개별 영화에 평점을 투표한 횟수
# m : 평점을 부여하기 위한 최소 투표 횟수
# R : 개별 영화에 대한 평균 평점
# C : 전체 영화에 대한 평균 평점
# m값은 전체 투표횟수에서 상위 60%에 해당하는 횟수를 기준으로 함
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [101]:
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m))*R)+((m/(m+v))*C)

movies['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)


In [102]:
movies[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [104]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [109]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies, genre_sim_sorted_ind, 'The Godfather', 20)
similar_movies[['title','vote_count','weighted_vote']]

# similar_movies[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866 3112 4041  588 3337
  3378  281 1663 1464 1149 2839]]


,title,vote_count,weighted_vote
2731,The Godfather: Part II,3338,8.079586
1847,GoodFellas,3128,7.976937
3866,City of God,1814,7.759693
1663,Once Upon a Time in America,1069,7.657811
883,Catch Me If You Can,3795,7.557097
281,American Gangster,1502,7.141396
4041,This Is England,363,6.739664
1149,American Hustle,2807,6.717525
1243,Mean Streets,345,6.626569
2839,Rounders,439,6.530427


In [ ]:
# 장르만으로 영화가 전달하는 많은 요소와 분위기, 그리고 개인이 좋아하는 성향을 반영하기에는 부족함
# 좋아하는 영화배우나 감독을 보고 영화를 선택하는 경우가 있음